In [654]:
# libs
import os
import pandas as pd
# show all the columns in pandas df
pd.set_option('display.max_columns', None)
import requests
from dotenv import load_dotenv #pip3 install python-dotenv
# load secrets
load_dotenv()
import math
import json
from IPython.display import JSON


In [655]:
# iex cloud - api data request
iex_key = os.getenv("IEX_API_KEY")
url = 'https://cloud.iexapis.com/stable/ref-data/symbols?token=' + iex_key
response = requests.get(url)
data = response.json()
symbols_df = pd.DataFrame(data)
# print 
symbols_df.head()

,symbol,exchange,exchangeSuffix,exchangeName,exchangeSegment,exchangeSegmentName,name,date,type,iexId,region,currency,isEnabled,figi,cik,lei
0,A,XNYS,,New York Stock Exchange Inc,XNYS,New York Stock Exchange Inc,Agilent Technologies Inc.,2024-02-21,cs,IEX_46574843354B2D52,US,USD,True,BBG000C2V3D6,0001090872,QUIX8Y7A2WP0XRMW7G29
1,AA,XNYS,,New York Stock Exchange Inc,XNYS,New York Stock Exchange Inc,Alcoa Corp,2024-02-21,cs,IEX_4238333734532D52,US,USD,True,BBG00B3T3HD3,0001675149,549300T12EZ1F6PWWU29
2,AAA,ARCX,,Nyse Arca,ARCX,Nyse Arca,Investment Managers Series Trust II - AXS Firs...,2024-02-21,et,IEX_5030314338392D52,US,USD,True,BBG01B0JRCS6,0001587982,549300SU7ER9OFETRU41
3,AAAU,BATS,,Cboe Bzx U S Equities Exchange,BATS,Cboe Bzx U S Equities Exchange,Goldman Sachs Physical Gold ETF Trust - Goldma...,2024-02-21,et,IEX_474B433136332D52,US,USD,True,BBG00LPXX872,0001708646,None
4,AACG,XNAS,,Nasdaq All Markets,XNMS,Nasdaq Nms Global Market,ATA Creativity Global - ADR,2024-02-21,ad,IEX_44595A4C53392D52,US,USD,True,BBG000V2S3P6,0001420529,None


In [656]:
# NYSE and NASDAQ symbols only (filter)
filtered_df = symbols_df[symbols_df['exchange'].isin(['XNYS', 'XNAS'])]

# select cols
filtered_df = filtered_df[['symbol', 'exchange', 'exchangeName']]

# filter df
print(filtered_df.head())
data_size = len(filtered_df.index)
print()
print(f"Data size: {data_size}") # get length of dataframe

  symbol exchange                 exchangeName
0      A     XNYS  New York Stock Exchange Inc
1     AA     XNYS  New York Stock Exchange Inc
4   AACG     XNAS           Nasdaq All Markets
5   AACI     XNAS           Nasdaq All Markets
6  AACIU     XNAS           Nasdaq All Markets

Data size: 7785


In [657]:
batch_size = 100
# Assuming filtered_df is a DataFrame that contains a 'symbol' column.
# Calculate total batches
total_batches = math.ceil(len(filtered_df) / batch_size)
print(f"Total batches: {total_batches}")

combined_data = []

# function -> fetch market cap data in batches
def fetch_market_cap(symbols_batch, iex_key):
    symbols_str = ','.join(symbols_batch)
    url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbols_str}&types=quote&token={iex_key}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {}
    
# Initialize a counter for completed batches
completed_batches = 0

# Define the length of the progress bar
progress_bar_length = 50

# batch processing - IMPORTANT (API efficiency)
for i in range(0, len(filtered_df), batch_size):
    batch_symbols = filtered_df['symbol'].iloc[i:i+batch_size].tolist()
    batch_data = fetch_market_cap(batch_symbols, iex_key)  # Make sure iex_key is defined and valid
    
    # process and append data for each symbol in the batch
    for symbol in batch_symbols:
        market_cap = batch_data.get(symbol, {}).get('quote', {}).get('marketCap', None)
        combined_data.append({'symbol': symbol, 'marketcap': market_cap})
    
    # Increment the completed batches counter
    completed_batches += 1

    # Calculate the progress
    progress = (completed_batches / total_batches)
    filled_length = int(round(progress_bar_length * progress))
    
    # Create the progress bar
    bar = '█' * filled_length + '-' * (progress_bar_length - filled_length)
    
    # Print the progress bar with the percentage
    print(f"\rProgress: |{bar}| {progress*100:.2f}% Complete", end="\r")

# Ensure the next print happens on the next line
print()

# convert combined data into a DataFrame
screener_df = pd.DataFrame(combined_data)

# join with the filtered_df - this adds exchange and exchangeName data
screener_df = screener_df.merge(filtered_df[['symbol', 'exchange', 'exchangeName']], on='symbol', how='left')

Total batches: 78
Progress: |██████████████████████████████████████████████████| 100.00% Complete


In [658]:
print(screener_df.head())
print()
print(f'Length: {len(screener_df.index)}')

  symbol     marketcap exchange                 exchangeName
0      A  3.965706e+10     XNYS  New York Stock Exchange Inc
1     AA  4.990075e+09     XNYS  New York Stock Exchange Inc
2   AACG  4.209672e+07     XNAS           Nasdaq All Markets
3   AACI  8.880890e+07     XNAS           Nasdaq All Markets
4  AACIU  8.848655e+07     XNAS           Nasdaq All Markets

Length: 7785


In [659]:
# drop NAs
screener_df.dropna(inplace=True)

# determins the count of droped NAs
print(f'New length: {len(screener_df)}')
print()

screener_df.head(5)

New length: 7679



,symbol,marketcap,exchange,exchangeName
0,A,3.965706e+10,XNYS,New York Stock Exchange Inc
1,AA,4.990075e+09,XNYS,New York Stock Exchange Inc
2,AACG,4.209672e+07,XNAS,Nasdaq All Markets
3,AACI,8.880890e+07,XNAS,Nasdaq All Markets
4,AACIU,8.848655e+07,XNAS,Nasdaq All Markets


In [660]:
batch_size = 100
total_batches = math.ceil(len(screener_df) / batch_size)
print(f"Total batches: {total_batches}")

quote_data_list = []

def fetch_quote_data(symbols_batch, iex_key):
    symbols_str = ','.join(symbols_batch)
    url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbols_str}&types=quote&token={iex_key}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {}
    
# initialize a counter for completed batches
completed_batches = 0

# define the length of the progress bar
progress_bar_length = 50

"""scalable endpoint variable data request"""
for i in range(0, len(screener_df), batch_size):
    batch_symbols = screener_df['symbol'].iloc[i:i+batch_size].tolist()
    batch_data = fetch_quote_data(batch_symbols, iex_key)
    
    # process and append data for each symbol in the batch
    for symbol in batch_symbols:
        quote_data = batch_data.get(symbol, {}).get('quote', {})
        quote_data_list.append({
            'symbol': symbol, # KEY
            'latestPrice': quote_data.get('latestPrice', None),
            'close': quote_data.get('close', None), 
            'previousClose': quote_data.get('previousClose', None),
            'extendedPrice': quote_data.get('extendedPrice', None), 
            'extendedChange': quote_data.get('extendedChange', None), 
            'extendedChangePercent': quote_data.get('extendedChangePercent', None), 
            'latestVolume': quote_data.get('latestVolume', None), 
            'volume': quote_data.get('volume', None), 
            'previousVolume': quote_data.get('previousVolume', None), 
            'primaryExchange': quote_data.get('primaryExchange', None),
            'avgTotalVolume': quote_data.get('avgTotalVolume', None),
            'calculationPrice': quote_data.get('calculationPrice', None),
            'change': quote_data.get('change', None),
            'changePercent': quote_data.get('changePercent', None),
            'companyName': quote_data.get('companyName', None),
            # IEX real time prices - after hours without UTP authorization (maybe)
            'iexClose': quote_data.get('iexClose', None),
            'iexCloseTime': quote_data.get('iexCloseTime', None),
            'iexRealtimePrice': quote_data.get('iexRealtimePrice', None),
            'iexLastUpdated': quote_data.get('iexLastUpdated', None),
            'iexVolume': quote_data.get('iexVolume', None),
            # updates overkill
            'latestTime': quote_data.get('latestTime', None),
            'latestUpdate': quote_data.get('latestUpdate', None),
            # add parameters 'isUSMarketOpen', 'closeSource', 'openSource', 'iexOpen'
            'isUSMarketOpen': quote_data.get('isUSMarketOpen', None),
            'closeSource': quote_data.get('closeSource', None),
            'openSource': quote_data.get('openSource', None),
            'iexOpen': quote_data.get('iexOpen', None)})
    
    # increment completed batches counter
    completed_batches += 1

    # calculate progress
    progress = (completed_batches / total_batches)
    filled_length = int(round(progress_bar_length * progress))
    
    # create progress bar
    bar = '█' * filled_length + '-' * (progress_bar_length - filled_length)
    
    # print progress bar with percentage
    print(f"\rProgress: |{bar}| {progress*100:.2f}% Complete", end="\r")

# Convert the combined data into a DataFrame
quote_df = pd.DataFrame(quote_data_list)

# MERGE DATA
screener_df = screener_df.merge(quote_df, on='symbol', how='left')

screener_df.head(5)

Total batches: 77


,symbol,marketcap,exchange,exchangeName,latestPrice,close,previousClose,extendedPrice,extendedChange,extendedChangePercent,latestVolume,volume,previousVolume,primaryExchange,avgTotalVolume,calculationPrice,change,changePercent,companyName,iexClose,iexCloseTime,iexRealtimePrice,iexLastUpdated,iexVolume,latestTime,latestUpdate,isUSMarketOpen,closeSource,openSource,iexOpen
0,A,3.965706e+10,XNYS,New York Stock Exchange Inc,135.3300,135.33,134.1400,134.51,-0.82,-0.00606,1134719,1134719,1563909.0,NEW YORK STOCK EXCHANGE INC.,1313924,close,1.19,0.00887,Agilent Technologies Inc.,135.34,1.708549e+12,135.34,1.708549e+12,40482.0,"February 21, 2024",1708549202457,False,official,official,133.57
1,AA,4.990075e+09,XNYS,New York Stock Exchange Inc,27.9600,27.96,26.6900,27.98,0.02,0.00072,5637108,5637108,4711849.0,NEW YORK STOCK EXCHANGE INC.,5707777,close,1.27,0.04758,Alcoa Corp,27.97,1.708549e+12,27.97,1.708549e+12,196693.0,"February 21, 2024",1708549436579,False,official,official,27.09
2,AACG,4.209672e+07,XNAS,Nasdaq All Markets,1.3406,NaN,1.3406,NaN,NaN,NaN,6388,6388,9245.0,NASDAQ,20179,previousclose,0.00,0.00000,ATA Creativity Global - ADR,1.46,1.708117e+12,0.00,0.000000e+00,0.0,"February 20, 2024",1708405200000,False,official,official,NaN
3,AACI,8.880890e+07,XNAS,Nasdaq All Markets,11.0200,NaN,11.0200,NaN,NaN,NaN,556,556,26.0,NASDAQ,13809,previousclose,0.00,0.00000,Armada Acquisition Corp I,10.93,1.707235e+12,0.00,0.000000e+00,0.0,"February 20, 2024",1708405200000,False,official,official,NaN
4,AACIU,8.848655e+07,XNAS,Nasdaq All Markets,10.9000,NaN,10.9000,NaN,NaN,NaN,1655,0,1655.0,NASDAQ,6983,previousclose,0.00,0.00000,Armada Acquisition Corp I - Units (1 Ord & 1/2...,10.90,1.706906e+12,0.00,0.000000e+00,0.0,"February 2, 2024",1706850000000,False,official,official,NaN


In [661]:
def fetch_shares_outstanding_data(symbols_batch, iex_key):
    symbols_str = ','.join(symbols_batch)
    url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbols_str}&types=stats&token={iex_key}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {}

# Assuming screener_df is a DataFrame containing the symbols
batch_size = 100
total_batches = math.ceil(len(screener_df) / batch_size)
print(f"Total batches: {total_batches}")

shares_outstanding_data_list = []

# initialize a counter for completed batches
completed_batches = 0
# define the length of the progress bar
progress_bar_length = 50

# Replace 'screener_df' with the actual DataFrame containing the symbols
for i in range(0, len(screener_df), batch_size):
    batch_symbols = screener_df['symbol'].iloc[i:i+batch_size].tolist()
    batch_data = fetch_shares_outstanding_data(batch_symbols, iex_key)
    
    # Process and append data for each symbol in the batch
    for symbol in batch_symbols:
        stats_data = batch_data.get(symbol, {}).get('stats', {})
        shares_outstanding_data_list.append({
            'symbol': symbol,
            'sharesOutstanding': stats_data.get('sharesOutstanding', None),
            'avg10Volume': stats_data.get('avg10Volume', None),
            'avg30Volume': stats_data.get('avg30Volume', None),
        })

    # increment completed batches counter
    completed_batches += 1

    # calculate progress
    progress = (completed_batches / total_batches)
    filled_length = int(round(progress_bar_length * progress))
    
    # create progress bar
    bar = '█' * filled_length + '-' * (progress_bar_length - filled_length)
    
    # print progress bar with percentage
    print(f"\rProgress: |{bar}| {progress*100:.2f}% Complete", end="\r")

# Convert the combined data into a DataFrame
shares_outstanding_df = pd.DataFrame(shares_outstanding_data_list)

# MERGE DATA
screener_df = screener_df.merge(shares_outstanding_df, on='symbol', how='left')

screener_df.head(5)

Total batches: 77


,symbol,marketcap,exchange,exchangeName,latestPrice,close,previousClose,extendedPrice,extendedChange,extendedChangePercent,latestVolume,volume,previousVolume,primaryExchange,avgTotalVolume,calculationPrice,change,changePercent,companyName,iexClose,iexCloseTime,iexRealtimePrice,iexLastUpdated,iexVolume,latestTime,latestUpdate,isUSMarketOpen,closeSource,openSource,iexOpen,sharesOutstanding,avg10Volume,avg30Volume
0,A,3.965706e+10,XNYS,New York Stock Exchange Inc,135.3300,135.33,134.1400,134.51,-0.82,-0.00606,1134719,1134719,1563909.0,NEW YORK STOCK EXCHANGE INC.,1313924,close,1.19,0.00887,Agilent Technologies Inc.,135.34,1.708549e+12,135.34,1.708549e+12,40482.0,"February 21, 2024",1708549202457,False,official,official,133.57,293039707,1302235,1313924
1,AA,4.990075e+09,XNYS,New York Stock Exchange Inc,27.9600,27.96,26.6900,27.98,0.02,0.00072,5637108,5637108,4711849.0,NEW YORK STOCK EXCHANGE INC.,5707777,close,1.27,0.04758,Alcoa Corp,27.97,1.708549e+12,27.97,1.708549e+12,196693.0,"February 21, 2024",1708549436579,False,official,official,27.09,178471908,5087162,5707777
2,AACG,4.209672e+07,XNAS,Nasdaq All Markets,1.3406,NaN,1.3406,NaN,NaN,NaN,6388,6388,9245.0,NASDAQ,20179,previousclose,0.00,0.00000,ATA Creativity Global - ADR,1.46,1.708117e+12,0.00,0.000000e+00,0.0,"February 20, 2024",1708405200000,False,official,official,NaN,31401405,36625,20179
3,AACI,8.880890e+07,XNAS,Nasdaq All Markets,11.0200,NaN,11.0200,NaN,NaN,NaN,556,556,26.0,NASDAQ,13809,previousclose,0.00,0.00000,Armada Acquisition Corp I,10.93,1.707235e+12,0.00,0.000000e+00,0.0,"February 20, 2024",1708405200000,False,official,official,NaN,8058884,461,13809
4,AACIU,8.848655e+07,XNAS,Nasdaq All Markets,10.9000,NaN,10.9000,NaN,NaN,NaN,1655,0,1655.0,NASDAQ,6983,previousclose,0.00,0.00000,Armada Acquisition Corp I - Units (1 Ord & 1/2...,10.90,1.706906e+12,0.00,0.000000e+00,0.0,"February 2, 2024",1706850000000,False,official,official,NaN,15000000,0,6983


In [662]:
"""THINK ABOUT MOVING THIS UP TO MAKE THE API CALLS MORE EFFICIENT"""
# print starting length
print(f'Length: {len(screener_df)}')
# filter out all negative changes from 'change'column
screener_df = screener_df[screener_df['change'] > 0]
# print new length after dropping the negative price movements
print(f'New length: {len(screener_df)}')
screener_df.head(5)

Length: 7679
New length: 2650


,symbol,marketcap,exchange,exchangeName,latestPrice,close,previousClose,extendedPrice,extendedChange,extendedChangePercent,latestVolume,volume,previousVolume,primaryExchange,avgTotalVolume,calculationPrice,change,changePercent,companyName,iexClose,iexCloseTime,iexRealtimePrice,iexLastUpdated,iexVolume,latestTime,latestUpdate,isUSMarketOpen,closeSource,openSource,iexOpen,sharesOutstanding,avg10Volume,avg30Volume
0,A,3.965706e+10,XNYS,New York Stock Exchange Inc,135.3300,135.33,134.14,134.51,-0.82,-0.00606,1134719,1134719,1563909.0,NEW YORK STOCK EXCHANGE INC.,1313924,close,1.1900,0.00887,Agilent Technologies Inc.,135.3400,1.708549e+12,135.3400,1.708549e+12,40482.0,"February 21, 2024",1708549202457,False,official,official,133.570,293039707,1302235,1313924
1,AA,4.990075e+09,XNYS,New York Stock Exchange Inc,27.9600,27.96,26.69,27.98,0.02,0.00072,5637108,5637108,4711849.0,NEW YORK STOCK EXCHANGE INC.,5707777,close,1.2700,0.04758,Alcoa Corp,27.9700,1.708549e+12,27.9700,1.708549e+12,196693.0,"February 21, 2024",1708549436579,False,official,official,27.090,178471908,5087162,5707777
9,AAGR,4.640341e+07,XNAS,Nasdaq All Markets,0.8019,NaN,0.80,NaN,NaN,NaN,56342,56342,134695.0,NASDAQ,113025,iexlasttrade,0.0019,0.00237,African Agriculture Holdings Inc,0.8019,1.708548e+12,0.8019,1.708548e+12,3327.0,"February 21, 2024",1708547875559,False,official,official,0.800,57866830,95723,113025
10,AAL,9.665865e+09,XNAS,Nasdaq All Markets,14.7900,NaN,14.70,NaN,NaN,NaN,22989655,22989655,30253071.0,NASDAQ,35653976,iexlasttrade,0.0900,0.00612,American Airlines Group Inc,14.7900,1.708549e+12,14.7900,1.708549e+12,224922.0,"February 21, 2024",1708549198459,False,official,official,14.625,653540550,28060970,35653976
11,AAME,5.253589e+07,XNAS,Nasdaq All Markets,2.5750,NaN,2.51,NaN,NaN,NaN,1520,1520,12449.0,NASDAQ,6801,iexlasttrade,0.0650,0.02590,Atlantic American Corp.,2.5750,1.708549e+12,2.5750,1.708549e+12,1.0,"February 21, 2024",1708549136844,False,official,official,2.575,20402288,5855,6801


In [663]:
# categorize market cap
def categorize_market_cap(x):
    if x < 300e6:  #  < 300 million
        return 'Micro-Cap'
    elif 300e6 <= x < 2e9:  # 300 million to 2 billion
        return 'Small-Cap'
    elif 2e9 <= x < 10e9:  # 2 billion to 10 billion
        return 'Mid-Cap'
    elif 10e9 <= x < 200e9:  # 10 billion to 200 billion
        return 'Large-Cap'
    elif x >= 200e9:  # >= 200 billion
        return 'Mega-Cap'
    else:
        return 'Unknown'  # handles negative or NaN values

screener_df.loc[:, 'marketcapType'] = screener_df['marketcap'].apply(categorize_market_cap)

screener_df.head(10)

,symbol,marketcap,exchange,exchangeName,latestPrice,close,previousClose,extendedPrice,extendedChange,extendedChangePercent,latestVolume,volume,previousVolume,primaryExchange,avgTotalVolume,calculationPrice,change,changePercent,companyName,iexClose,iexCloseTime,iexRealtimePrice,iexLastUpdated,iexVolume,latestTime,latestUpdate,isUSMarketOpen,closeSource,openSource,iexOpen,sharesOutstanding,avg10Volume,avg30Volume,marketcapType
0,A,3.965706e+10,XNYS,New York Stock Exchange Inc,135.3300,135.33,134.14,134.51,-0.82,-0.00606,1134719,1134719,1563909.0,NEW YORK STOCK EXCHANGE INC.,1313924,close,1.1900,0.00887,Agilent Technologies Inc.,135.3400,1.708549e+12,135.3400,1.708549e+12,40482.0,"February 21, 2024",1708549202457,False,official,official,133.570,293039707,1302235,1313924,Large-Cap
1,AA,4.990075e+09,XNYS,New York Stock Exchange Inc,27.9600,27.96,26.69,27.98,0.02,0.00072,5637108,5637108,4711849.0,NEW YORK STOCK EXCHANGE INC.,5707777,close,1.2700,0.04758,Alcoa Corp,27.9700,1.708549e+12,27.9700,1.708549e+12,196693.0,"February 21, 2024",1708549436579,False,official,official,27.090,178471908,5087162,5707777,Mid-Cap
9,AAGR,4.640341e+07,XNAS,Nasdaq All Markets,0.8019,NaN,0.80,NaN,NaN,NaN,56342,56342,134695.0,NASDAQ,113025,iexlasttrade,0.0019,0.00237,African Agriculture Holdings Inc,0.8019,1.708548e+12,0.8019,1.708548e+12,3327.0,"February 21, 2024",1708547875559,False,official,official,0.800,57866830,95723,113025,Micro-Cap
10,AAL,9.665865e+09,XNAS,Nasdaq All Markets,14.7900,NaN,14.70,NaN,NaN,NaN,22989655,22989655,30253071.0,NASDAQ,35653976,iexlasttrade,0.0900,0.00612,American Airlines Group Inc,14.7900,1.708549e+12,14.7900,1.708549e+12,224922.0,"February 21, 2024",1708549198459,False,official,official,14.625,653540550,28060970,35653976,Mid-Cap
11,AAME,5.253589e+07,XNAS,Nasdaq All Markets,2.5750,NaN,2.51,NaN,NaN,NaN,1520,1520,12449.0,NASDAQ,6801,iexlasttrade,0.0650,0.02590,Atlantic American Corp.,2.5750,1.708549e+12,2.5750,1.708549e+12,1.0,"February 21, 2024",1708549136844,False,official,official,2.575,20402288,5855,6801,Micro-Cap
14,AAON,6.726453e+09,XNAS,Nasdaq All Markets,81.9300,NaN,81.75,NaN,NaN,NaN,396290,396290,566898.0,NASDAQ,416763,iexlasttrade,0.1800,0.00220,AAON Inc.,81.9300,1.708549e+12,81.9300,1.708549e+12,12223.0,"February 21, 2024",1708549192702,False,official,official,81.800,82100000,467100,416763,Mid-Cap
17,AAPD,3.656702e+07,XNAS,Nasdaq All Markets,21.5100,NaN,21.50,NaN,NaN,NaN,121125,121125,294141.0,NASDAQ,259953,iexlasttrade,0.0100,0.00047,Direxion Shares ETF Trust - Direxion Daily AAP...,21.5100,1.708548e+12,21.5100,1.708548e+12,230.0,"February 21, 2024",1708547597232,False,official,official,21.410,1700001,275306,259953,Micro-Cap
18,AAPL,2.815364e+12,XNAS,Nasdaq All Markets,182.3200,NaN,181.56,NaN,NaN,NaN,41354837,41354837,53665553.0,NASDAQ,54791161,iexlasttrade,0.7600,0.00419,Apple Inc,182.3200,1.708549e+12,182.2000,1.708552e+12,524687.0,"February 21, 2024",1708549199996,False,official,official,181.995,15441881000,53632416,54791161,Mega-Cap
19,AAPU,3.437502e+07,XNAS,Nasdaq All Markets,25.0000,NaN,24.84,NaN,NaN,NaN,67742,67742,119063.0,NASDAQ,166757,iexlasttrade,0.1600,0.00644,Direxion Shares ETF Trust - Direxion Daily AAP...,25.0000,1.708549e+12,25.0000,1.708549e+12,589.0,"February 21, 2024",1708549168439,False,official,official,24.970,1375001,156476,166757,Micro-Cap
20,AAT,1.331182e+09,XNYS,New York Stock Exchange Inc,21.8600,21.86,21.63,21.86,0.00,0.00000,587262,587262,206523.0,NEW YORK STOCK EXCHANGE INC.,339066,close,0.2300,0.01063,American Assets Trust Inc,21.8700,1.708549e+12,21.8700,1.708549e+12,6378.0,"February 21, 2024",1708549202363,False,official,official,21.560,60895786,286259,339066,Small-Cap


In [664]:
# # calculate iexChange and iexChangePercent - custom calculation
# screener_df['u_iexChange'] = screener_df['iexClose'] - screener_df['close']
# screener_df['u_iexChangePercent'] = (screener_df['iexChange'] / screener_df['close'])

In [665]:
# print columns for screener_df
print(screener_df.columns)

Index(['symbol', 'marketcap', 'exchange', 'exchangeName', 'latestPrice',
       'close', 'previousClose', 'extendedPrice', 'extendedChange',
       'extendedChangePercent', 'latestVolume', 'volume', 'previousVolume',
       'primaryExchange', 'avgTotalVolume', 'calculationPrice', 'change',
       'changePercent', 'companyName', 'iexClose', 'iexCloseTime',
       'iexRealtimePrice', 'iexLastUpdated', 'iexVolume', 'latestTime',
       'latestUpdate', 'isUSMarketOpen', 'closeSource', 'openSource',
       'iexOpen', 'sharesOutstanding', 'avg10Volume', 'avg30Volume',
       'marketcapType'],
      dtype='object')


RESET FROM HERE

In [709]:
# create a columns filter
screener_cols = ['symbol', # KEY
                 # market cap
                'marketcap',
                'marketcapType', 
                # price(s)
                'close', 
                'latestPrice', 
                'extendedPrice',
                # relative change
                'calculationPrice',
                'change', #mandatory
                'changePercent', 
                # shares (approx. float)
                'sharesOutstanding', 
                # volume
                'latestVolume',
                'avgTotalVolume', # NEW ############################ 
                'avg10Volume',
                'avg30Volume',
                'iexVolume', # NEW ############################
                #info
                'isUSMarketOpen',
                'exchange',
                'exchangeName',
                'companyName']
                # 'closeSource',
                # 'openSource',
                # IEX real time prices - after hours without UTP authorization
                # 'iexClose',
                # 'iexCloseTime',
                # 'iexRealtimePrice',
                # 'u_iexChange',
                # 'u_iexChangePercent',
                # 'iexOpen'] 
                #### ADD -  CHANGE % (CALCULATION - custom) ####
            

screener_df_x = screener_df[screener_cols]

screener_df_x.head(25)

,symbol,marketcap,marketcapType,close,latestPrice,extendedPrice,calculationPrice,change,changePercent,sharesOutstanding,latestVolume,avgTotalVolume,avg10Volume,avg30Volume,iexVolume,isUSMarketOpen,exchange,exchangeName,companyName
0,A,3.965706e+10,Large-Cap,135.33,135.3300,134.51,close,1.1900,0.00887,293039707,1134719,1313924,1302235,1313924,40482.0,False,XNYS,New York Stock Exchange Inc,Agilent Technologies Inc.
1,AA,4.990075e+09,Mid-Cap,27.96,27.9600,27.98,close,1.2700,0.04758,178471908,5637108,5707777,5087162,5707777,196693.0,False,XNYS,New York Stock Exchange Inc,Alcoa Corp
9,AAGR,4.640341e+07,Micro-Cap,NaN,0.8019,NaN,iexlasttrade,0.0019,0.00237,57866830,56342,113025,95723,113025,3327.0,False,XNAS,Nasdaq All Markets,African Agriculture Holdings Inc
10,AAL,9.665865e+09,Mid-Cap,NaN,14.7900,NaN,iexlasttrade,0.0900,0.00612,653540550,22989655,35653976,28060970,35653976,224922.0,False,XNAS,Nasdaq All Markets,American Airlines Group Inc
11,AAME,5.253589e+07,Micro-Cap,NaN,2.5750,NaN,iexlasttrade,0.0650,0.02590,20402288,1520,6801,5855,6801,1.0,False,XNAS,Nasdaq All Markets,Atlantic American Corp.
14,AAON,6.726453e+09,Mid-Cap,NaN,81.9300,NaN,iexlasttrade,0.1800,0.00220,82100000,396290,416763,467100,416763,12223.0,False,XNAS,Nasdaq All Markets,AAON Inc.
17,AAPD,3.656702e+07,Micro-Cap,NaN,21.5100,NaN,iexlasttrade,0.0100,0.00047,1700001,121125,259953,275306,259953,230.0,False,XNAS,Nasdaq All Markets,Direxion Shares ETF Trust - Direxion Daily AAP...
18,AAPL,2.815364e+12,Mega-Cap,NaN,182.3200,NaN,iexlasttrade,0.7600,0.00419,15441881000,41354837,54791161,53632416,54791161,524687.0,False,XNAS,Nasdaq All Markets,Apple Inc
19,AAPU,3.437502e+07,Micro-Cap,NaN,25.0000,NaN,iexlasttrade,0.1600,0.00644,1375001,67742,166757,156476,166757,589.0,False,XNAS,Nasdaq All Markets,Direxion Shares ETF Trust - Direxion Daily AAP...
20,AAT,1.331182e+09,Small-Cap,21.86,21.8600,21.86,close,0.2300,0.01063,60895786,587262,339066,286259,339066,6378.0,False,XNYS,New York Stock Exchange Inc,American Assets Trust Inc


In [710]:
# drop all NaN when 'close' is NaN
print(f'Length: {len(screener_df_x)}')
screener_df_x = screener_df_x.dropna(subset=['close'])
print(f'New Length: {len(screener_df_x)}')

Length: 2650
New Length: 1404


In [711]:
# create five (5) dataframes for each marketcap type
microcap_df = screener_df_x[screener_df_x['marketcapType'] == 'Micro-Cap']
smallcap_df = screener_df_x[screener_df_x['marketcapType'] == 'Small-Cap']
midcap_df = screener_df_x[screener_df_x['marketcapType'] == 'Mid-Cap']
largecap_df = screener_df_x[screener_df_x['marketcapType'] == 'Large-Cap']
megacap_df = screener_df_x[screener_df_x['marketcapType'] == 'Mega-Cap']

In [669]:
# # print
# largecap_df.head(5)

In [670]:
# get columns
# print(largecap_df.columns)

In [671]:
# largecap_df['changePercent']

In [712]:
def filter_stocks(df, 
                  market_cap_type=None, 
                  price_min=None, 
                  change_min_percent=None,  # Pre-market gap percentage
                  volume_min=None, 
                  volume_avg_comparison=None,  # '10day' or '30day'
                  shares_outstanding_min=None, 
                  shares_outstanding_max=None,
                  extended_price_min=None):  # For pre-market gap analysis
    # Market Cap Type
    if market_cap_type is not None:
        df = df[df['marketcapType'] == market_cap_type]

    # Change Percent
    if change_min_percent is not None:
        df = df[df['changePercent'] >= change_min_percent]
    
    # PRICE
    if price_min is not None:
        df = df[df['latestPrice'] >= price_min]
    
    # Ensure 'close' and 'extendedPrice' are not null to avoid division by zero or null comparisons
    df = df.dropna(subset=['close', 'extendedPrice'])
    
    # Filter for gap-ups: only include stocks where extendedPrice > close
    df = df[df['extendedPrice'] > df['close']]
    
    # VOLUME
    if volume_min is not None:
        df = df[df['latestVolume'] >= volume_min]
    
    # Dynamic Volume Comparison
    if volume_avg_comparison is not None:
        if volume_avg_comparison == '10day':
            df = df[df['latestVolume'] >= df['avg10Volume']]
        elif volume_avg_comparison == '30day':
            df = df[df['latestVolume'] >= df['avg30Volume']]
    
    # SHARES OUTSTANDING
    if shares_outstanding_min is not None:
        df = df[df['sharesOutstanding'] >= shares_outstanding_min]
    if shares_outstanding_max is not None:
        df = df[df['sharesOutstanding'] <= shares_outstanding_max]
    
    # EXTENDED PRICE (For pre-market gap analysis)
    if extended_price_min is not None:
        df = df[df['extendedPrice'] >= extended_price_min]
    
    return df


In [713]:
#### MICROCAP FILTER ####
microcap_df = filter_stocks(
    microcap_df, 
    market_cap_type='Micro-Cap', 
    price_min=0.50,  # Keep minimum price to filter out ultra-low-priced stocks
    change_min_percent=.03,  # Increase to target more significant gaps; adjusted from 0.02 to 3%
    volume_min=10000,  # New: Set a minimum volume threshold to ensure liquidity
    volume_avg_comparison='10day'  # Ensure current volume is above the 10-day average
)

# Order descending by changePercent to prioritize highest gap-ups
microcap_df = microcap_df.sort_values('changePercent', ascending=False)

# After sorting, re-index the dataframe for clarity
microcap_df = microcap_df.reset_index(drop=True)

# # get first 10 and inplace the dataframe
# microcap_df = microcap_df.iloc[:10] 

# Print the length of the dataframe to confirm the number of targeted stocks
print(f"Length: {len(microcap_df)}")

microcap_df

Length: 5


,symbol,marketcap,marketcapType,close,latestPrice,extendedPrice,calculationPrice,change,changePercent,sharesOutstanding,latestVolume,avgTotalVolume,avg10Volume,avg30Volume,iexVolume,isUSMarketOpen,exchange,exchangeName,companyName
0,KUKE,87192505.0,Micro-Cap,2.09,2.09,2.100,iexlasttrade,0.14,0.07179,41718902,59473,79403,58597,79403,406.0,False,XNYS,New York Stock Exchange Inc,Kuke Music Holding Ltd - ADR
1,CZOO,179384330.0,Micro-Cap,4.64,4.64,4.765,iexlasttrade,0.29,0.06667,38660416,29886,33786,14260,33786,100.0,False,XNYS,New York Stock Exchange Inc,Cazoo Group Ltd - Class A
2,SJT,239569211.0,Micro-Cap,5.14,5.14,5.190,close,0.28,0.05761,46608796,483430,352852,351484,352852,7329.0,False,XNYS,New York Stock Exchange Inc,San Juan Basin Royalty Trust - Unit
3,AKA,112648426.0,Micro-Cap,10.60,10.60,11.150,iexlasttrade,0.56,0.05578,10627210,13463,7307,5773,7307,74.0,False,XNYS,New York Stock Exchange Inc,a.k.a. Brands Holding Corp
4,ORN,207943104.0,Micro-Cap,6.40,6.40,6.449,close,0.20,0.03226,32491110,120236,96171,104650,96171,7645.0,False,XNYS,New York Stock Exchange Inc,Orion Group Holdings Inc


In [674]:
#### SMALL-CAP FILTER ####
# Filtering Small-Cap Stocks
smallcap_df = filter_stocks(
    df=smallcap_df,  # Your DataFrame of Small-Cap stocks
    market_cap_type='Small-Cap',
    price_min=1,  # Minimum price to filter out penny stocks but allow for small-cap volatility
    change_min_percent=.02,  # Looking for at least a 2% pre-market gap up to account for small-cap volatility
    volume_min=20000,  # Setting a volume minimum to ensure some liquidity
    volume_avg_comparison='10day'  # Comparing to 10-day average volume to gauge significant interest
)

# Order descending by changePercent to prioritize stocks with the highest gap-ups
smallcap_df = smallcap_df.sort_values('changePercent', ascending=False)

# After sorting, re-index the dataframe for clarity
smallcap_df = smallcap_df.reset_index(drop=True)

# Limiting the output for a focused review, you can adjust the count as needed
smallcap_df = smallcap_df.iloc[:10]

# Print the length of the dataframe to confirm the targeted number of stocks
print(f"Length: {len(smallcap_df)}")

# Display the filtered Small-Cap stocks
smallcap_df


In [716]:
#### MID-CAP FILTER ####
# Filtering Mid-Cap Stocks
midcap_df_filtered = filter_stocks(
    df=midcap_df,  # Assuming midcap_df is your DataFrame of Mid-Cap stocks
    market_cap_type='Mid-Cap',
    price_min=5,  # Adjust as needed, setting a floor to avoid very low-priced stocks
    change_min_percent=.015,  # Targeting a 1.5% pre-market gap up as significant for mid-caps
    volume_min=50000,  # Setting a volume minimum to ensure liquidity and interest
    volume_avg_comparison='10day'  # Comparing to 10-day average volume for significant interest indication
)

# Order descending by changePercent to prioritize stocks with the highest gap-ups
midcap_df_filtered = midcap_df_filtered.sort_values('changePercent', ascending=False)

# After sorting, re-index the dataframe for clarity
midcap_df_filtered = midcap_df_filtered.reset_index(drop=True)

# Limiting the output for a focused review, adjust the count as per your strategy requirements
midcap_df_filtered = midcap_df_filtered.iloc[:10]

# Print the length of the dataframe to confirm the targeted number of stocks
print(f"Length: {len(midcap_df_filtered)}")

# Display the filtered Mid-Cap stocks
midcap_df_filtered


Length: 0


,symbol,marketcap,marketcapType,close,latestPrice,extendedPrice,calculationPrice,change,changePercent,sharesOutstanding,latestVolume,avgTotalVolume,avg10Volume,avg30Volume,iexVolume,isUSMarketOpen,exchange,exchangeName,companyName


In [714]:
#### LARGE-CAP FILTER ####
largecap_df = filter_stocks(
    df=largecap_df,  # Your DataFrame of Large-Cap stocks
    market_cap_type='Large-Cap',
    price_min=10,  # Adjust as needed for large-cap stocks
    change_min_percent=.01,  # Looking for at least a 0.5% pre-market gap up
    volume_min=50000,  # Setting a volume minimum to ensure liquidity
    volume_avg_comparison='10day'  # Comparing to 10-day average volume to gauge interest
)

# Order descending by changePercent to prioritize highest gap-ups
largecap_df = largecap_df.sort_values('changePercent', ascending=False)

# After sorting, re-index the dataframe for clarity
largecap_df = largecap_df.reset_index(drop=True)

# # get first 10 and inplace the dataframe
# largecap_df = largecap_df.iloc[:10] 

# Print the length of the dataframe to confirm the number of targeted stocks
print(f"Length: {len(largecap_df)}")

largecap_df

Length: 25


,symbol,marketcap,marketcapType,close,latestPrice,extendedPrice,calculationPrice,change,changePercent,sharesOutstanding,latestVolume,avgTotalVolume,avg10Volume,avg30Volume,iexVolume,isUSMarketOpen,exchange,exchangeName,companyName
0,GRMN,2.555801e+10,Large-Cap,133.58,133.58,135.00,close,10.83,0.08823,191331077,2560967,753581,909507,753581,74662.0,False,XNYS,New York Stock Exchange Inc,Garmin Ltd
1,TOL,1.122565e+10,Large-Cap,107.64,107.64,108.70,close,4.09,0.03950,104288855,3906948,1400309,1873001,1400309,119141.0,False,XNYS,New York Stock Exchange Inc,Toll Brothers Inc.
2,BABA,1.939196e+11,Large-Cap,75.58,75.58,75.75,close,2.44,0.03336,2565752214,23467410,23805991,14558998,23805991,328491.0,False,XNYS,New York Stock Exchange Inc,Alibaba Group Holding Ltd - ADR
3,RACE,7.431853e+10,Large-Cap,402.27,402.27,406.75,close,11.82,0.03027,184747890,351465,395118,224337,395118,11495.0,False,XNYS,New York Stock Exchange Inc,Ferrari N.V.
4,EQNR,7.690951e+10,Large-Cap,25.61,25.61,25.67,close,0.67,0.02686,3003104605,4825194,4235537,4637850,4235537,154220.0,False,XNYS,New York Stock Exchange Inc,Equinor ASA - ADR
5,IR,3.600264e+10,Large-Cap,88.94,88.94,89.15,close,2.20,0.02536,404796976,3305094,3091636,2909162,3091636,171848.0,False,XNYS,New York Stock Exchange Inc,Ingersoll-Rand Inc
6,EC,2.448499e+10,Large-Cap,11.91,11.91,11.93,close,0.29,0.02496,2055834735,1654026,1523607,1524226,1523607,55802.0,False,XNYS,New York Stock Exchange Inc,Ecopetrol SA - ADR
7,AEE,1.877428e+10,Large-Cap,71.40,71.40,71.61,close,1.53,0.02190,262945048,2469211,2010951,1969146,2010951,80092.0,False,XNYS,New York Stock Exchange Inc,Ameren Corp.
8,EIX,2.606349e+10,Large-Cap,67.95,67.95,67.99,close,1.41,0.02119,383568713,2136146,3161080,1986797,3161080,60325.0,False,XNYS,New York Stock Exchange Inc,Edison International
9,MRO,1.384110e+10,Large-Cap,23.65,23.65,23.90,close,0.48,0.02072,585247358,12215795,9273757,10508555,9273757,430730.0,False,XNYS,New York Stock Exchange Inc,Marathon Oil Corporation


In [715]:
#### MEGA-CAP FILTER ####
# Filtering Mega-Cap Stocks
megacap_df = filter_stocks(
    df=megacap_df,  # Assuming megacap_df is your DataFrame of Mega-Cap stocks
    market_cap_type='Mega-Cap',
    price_min=10,  # Mega-cap stocks are rarely below this price, but adjust as needed
    change_min_percent=.005,  # Adjusted for the lower volatility of mega-cap stocks
    volume_min=100000,  # A higher volume minimum to ensure liquidity and significant interest
    volume_avg_comparison='10day'  # Comparing to 10-day average volume to gauge interest
)

# Order descending by changePercent to prioritize highest gap-ups
megacap_df = megacap_df.sort_values('changePercent', ascending=False)

# After sorting, re-index the dataframe for clarity
megacap_df = megacap_df.reset_index(drop=True)

# Get the first 10 for an initial overview, or adjust as needed based on strategy
megacap_df = megacap_df.iloc[:10]

# Print the length of the dataframe to confirm the number of targeted stocks
print(f"Length: {len(megacap_df)}")

# Display the filtered Mega-Cap stocks
megacap_df


Length: 1


,symbol,marketcap,marketcapType,close,latestPrice,extendedPrice,calculationPrice,change,changePercent,sharesOutstanding,latestVolume,avgTotalVolume,avg10Volume,avg30Volume,iexVolume,isUSMarketOpen,exchange,exchangeName,companyName
0,MA,4.249532e+11,Mega-Cap,459.05,459.05,459.15,close,7.3,0.01616,925723131,2895787,2664115,2673462,2664115,90183.0,False,XNYS,New York Stock Exchange Inc,Mastercard Incorporated - Class A


In [678]:
# ADD - relative volume (ratio) data???
# some of this you might have already covered, but might be able to get more granular with the data

In [679]:
# create daily watchlist dataframe
# combine the filtered dataframes

In [680]:
# news

In [681]:
# news international

In [682]:
# sentiment - custom

In [683]:
# RBV - robust value score - custom

In [684]:
# key levels from historical data
# other data

In [685]:
# exogenous data

In [686]:
# do work with BENZINGA BZ squawk box - see if there is an API

In [687]:
# RSI, MACD, etc. - custom

In [688]:
# ichimoku cloud - custom

END - END - END - IN PROGRESS - END - END - END END - END - END - IN PROGRESS - END - END - END END - END - END - IN PROGRESS - END - END - END


In [689]:
# YAHOO data - testing

# data feed testing
import yfinance as yf
import pandas as pd

# List of stock symbols
symbols = ['PLTR', 'TSLA','NVDA']

# Fetch data
data = []
for symbol in symbols:
    stock = yf.Ticker(symbol)
    hist = stock.history(period="1d")  # Get the last day's data
    info = stock.info  # Get general stock info
    
    # Extracting the required information
    data.append({
        'Symbol': symbol,
        'Close Price': hist['Close'].iloc[-1] if not hist.empty else None,
        'Volume': hist['Volume'].iloc[-1] if not hist.empty else None,
        'Average Volume': info.get('averageVolume'),
        # get after hours current price
        'After Hours Price': info.get('postMarketPrice'),
        'Change in Price': info.get('regularMarketChange'),
        'Change in Percentage': info.get('regularMarketChangePercent'),
    })

# Convert to DataFrame
df = pd.DataFrame(data)

print(df)


  Symbol  Close Price     Volume  Average Volume After Hours Price  \
0   PLTR    22.740000   55339425        69471570              None   
1   TSLA   194.770004  102998068       111465968              None   
2   NVDA   674.719971   54866359        44840920              None   

  Change in Price Change in Percentage  
0            None                 None  
1            None                 None  
2            None                 None  


In [690]:
"""
IMPORTANT: Need UTP autorization to continue
https://iexcloud.io/documentation/using-core-data/getting-nasdaq-listed-utp-otc-stock-data.html
Step 1: https://www.utpplan.com/DOC/VendorAgreement.pdf
Step 2: https://www.utpplan.com/datafeed_approval
Step 3: https://www.utpplan.com/system_application

"""

# create a dataframe from utp_auth_columns.xlsx
utp_auth_df = pd.read_excel('utp_auth_columns.xlsx')
# replace NA with " " 
utp_auth_df = utp_auth_df.fillna(" ")

utp_auth_df

,utp_auth_required,humbled_trader_TARGETs,iex_real-time_alternatives,u_screener_df_columns_ACTIVE
0,close,Symbol,,
1,closeTime,Price ($),,
2,delayedPrice,Float (shares),,
3,delayedPriceTime,Change Close (%),,
4,extendedPrice,Volume Today (shares),,
5,extendedPriceTime,Avereage Volume (5day) (shares/day),,
6,extendedChange,Market Cap ($),,
7,extendedChangePercent,Held By Institutions (%),,
8,high,Sector,,
9,low,Company Name,,


In [691]:
# # DEV - marketcap groupby count
# marketcap_counts = screener_df['marketcapType'].value_counts()

# # convert SERIES to DF
# marketcap_counts_df = marketcap_counts.reset_index()
# marketcap_counts_df.columns = ['marketcapType', 'count']

# # add percentage column
# total_count = marketcap_counts.sum()
# marketcap_counts_df['percentage'] = (marketcap_counts_df['count'] / total_count) * 100

# # print
# marketcap_counts_df

In [692]:
# # DEV - checker
# total_count_CHECK = marketcap_counts_df['count'].sum()

# # if statement comparing the two values, return 'Data is correct' if they match, otherwise return 'Data is incorrect'
# if total_count_CHECK == len(screener_df):
#     print('Data lengths match! SUCCESS')
# else:    
#     print('Data mismatch! ERROR')
#     print()
#     print(f'Code checker: {total_count_CHECK}')
#     print(f'Data length: {len(screener_df)}')

In [693]:
# def print_available_stats_parameters(symbol, iex_key):
#     url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={iex_key}'
#     response = requests.get(url)
#     if response.status_code == 200:
#         data = response.json()
#         print(json.dumps(data, indent=4, sort_keys=True))
#     else:
#         print(f"Failed to fetch stats for {symbol}. Status code: {response.status_code}")

# pltr = 'PLTR'
# print_available_stats_parameters(pltr, iex_key)

In [694]:
# def fetch_deep_trades_data(symbols_batch, iex_key):
#     symbols_str = ','.join(symbols_batch)
#     url = f'https://cloud.iexapis.com/stable/deep/trades?symbols={symbols_str}&token={iex_key}'
#     response = requests.get(url)
#     if response.status_code == 200:
#         return response.json()
#     else:
#         return {}

# # Assuming screener_df is a DataFrame containing the symbols
# batch_size = 100
# # Example for illustrative purposes
# # screener_df = pd.DataFrame({'symbol': ['AAPL', 'MSFT', 'GOOGL']})
# total_batches = math.ceil(len(screener_df) / batch_size)
# print(f"Total batches: {total_batches}")

# deep_data_list = []

# for i in range(0, len(screener_df), batch_size):
#     batch_symbols = screener_df['symbol'].iloc[i:i+batch_size].tolist()
#     batch_data = fetch_deep_trades_data(batch_symbols, iex_key)
    
#     # Process and infer volume data for each symbol in the batch
#     for symbol in batch_symbols:
#         if symbol in batch_data and batch_data[symbol]:
#             volume = sum(trade['size'] for trade in batch_data[symbol])
#             deep_data_list.append({
#                 'symbol': symbol,
#                 'volume': volume,
#             })

# # Convert the combined data into a DataFrame
# iex_deep_df = pd.DataFrame(deep_data_list)

# print(iex_deep_df.head(20)) # data check
# print(len(iex_deep_df.index)) # data check

In [695]:
# print(f"screener_df column count: {len(screener_df.columns)}") # col count
# screener_df = screener_df.merge(shares_outstanding_df, on='symbol', how='left')

In [696]:
# # My Watchlist
# watchlist = ['PLTR', 'TSLA', 'NOW', 'SNOW','FB', 'NVDA', 'PYPL', 'ADBE', 'NFLX']
# watchlist_df = screener_df[screener_df['symbol'].isin(watchlist)]

# watchlist_df

In [697]:
# # DEV - API endpoing parameter availability. 
# schema_base_test_url = 'https://cloud.iexapis.com/stable'
# test_symbol = 'PLTR' # test ticker symbol
# schema_test_url = f"{schema_base_test_url}/stock/{test_symbol}/quote?schema=true&token={iex_key}"
# # GET request
# schema_test_response = requests.get(schema_test_url)

# print(json.dumps(schema_test_response.json(), indent=4))


IDEAS
1. Robust value score (RBV)
2. create data frames for each market cap
3. create a function that does math and querying based on rules / conditions
4. apply function to the market cap data sets
5. Look at other STATS now that the API is connected for the sharesOutstanding

NOTES
1. Float
1. Free Flt Mkt Cap
1. Free Flt
1. Shares outstanding

MANDATORY FIELDS
1. 